### Demonstration Overview: Translating a VRS (version 1.3) object into an Allele Profile

This notebook demonstrates the translation of GA4GH VRS (Version 1.3) into the HL7 FHIR Allele format. At the time of development, VRS 1.3 was the stable version of the model. However, the VRS community is actively working toward establishing VRS 2.0 as the next stable release. For more details on VRS, refer to the official [VRS Documentation](https://vrs.ga4gh.org/en/1.3/).  

In this notebook, we use example data to generate VRS objects with the `vrs-python` library. Once created, these objects can be converted into FHIR-compliant Allele using the `vrs_allele_to_allele_profile` method from the `VrsFhirAlleleTranslation` class. This method enables a one-way transformation from VRS to FHIR Allele.

### Prerequisites and Setup

To support the one way transformation from VRS Allele to FHIR Alelle of this demonstration, we set up the environment by importing the necessary libraries and modules. These include:

1. **External Package**:
   - `models` from `ga4gh.vrs`: Provides foundational data models for working with GA4GH Variation Representation Schema (VRS).

2. **Custom Project Modules**:
   - `AlleleNormalizer` from `normalize.allele_normalizer`: A utility for normalizing a vrs allele object.
   - `VrsFhirAlleleTranslation` from `translators.allele_translator`: A translation component for converting VRS alleles into FHIR Allele.

In [10]:
# importing the vrs models
from ga4gh.vrs import models
from translators.allele_translator import VrsFhirAlleleTranslation
from normalizers.allele_normalizer import AlleleNormalizer

normalize = AlleleNormalizer()
alleleTranslator = VrsFhirAlleleTranslation()

### Example 1: Creating, Normalizing, and Translating a VRS Allele

In this example, we first generate a VRS allele object and normalize it. Next, we translate the normalized allele into a FHIR-compliant Allele.

In [11]:
# Creating the VRS object and then normalizing it.
# Example 1 - Deletion origin: "NC_000001.11:g.1014265del"
start = 1014263
end = 1014265
refseq = "NC_000001.11"
alt_seq = "C"

interval = models.SequenceInterval(
    start=models.Number(value=start),
    end=models.Number(value=end)
)
location = models.SequenceLocation(
    sequence_id=f"refseq:{refseq}",
    interval=interval
)

state = models.LiteralSequenceExpression(sequence=alt_seq)
del_example_1 = models.Allele(location=location, state=state)
norm_del_example_1 = normalize.post_normalize_allele(del_example_1)
norm_del_example_1.as_dict()


{'_id': 'ga4gh:VA.BmF3zr2l6XLpLaK8GInM6Q3Emc3JyPD3',
 'type': 'Allele',
 'location': {'_id': 'ga4gh:VSL.i6Of9s2jVDuJ4vwU6sCeG-jT7ygmlfx6',
  'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 1014263},
   'end': {'type': 'Number', 'value': 1014265}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'C'}}

In [12]:
# Translating the normalized allele into an allele profile
allele_profile_del_example = alleleTranslator.vrs_allele_to_allele_profile(norm_del_example_1)

print(type(allele_profile_del_example))
allele_profile_del_example.model_dump()

<class 'profiles.allele.Allele'>


{'resourceType': 'MolecularDefinition',
 'contained': [{'resourceType': 'MolecularDefinition',
   'id': 'ref-to-nc000001',
   'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
      'code': 'dna',
      'display': 'DNA Sequence'}]},
   'representation': [{'code': [{'coding': [{'system': 'http://www.ncbi.nlm.nih.gov/refseq',
         'code': 'NC_000001.11'}]}]}]}],
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': '#ref-to-nc000001',
     'type': 'MolecularDefinition'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}]}},
     'startQuantity': {'value': 1014263.0},
     'endQuantity': {'value': 1014265.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/mol

### Example 2 

In [13]:
# Creating the VRS object and then normalizing it.
# Example 3 - Insertion origin: "NC_000001.11:g.113901365_113901366insATA"
start = 113901365
end = 113901365
refseq = "NC_000001.11"
alt_seq = "ATA"

interval = models.SequenceInterval(
    start=models.Number(value=start),
    end=models.Number(value=end)
)
location = models.SequenceLocation(
    sequence_id=f"refseq:{refseq}",
    interval=interval
)

state = models.LiteralSequenceExpression(sequence=alt_seq)
insertion_example= models.Allele(location=location, state=state)
norm_insertion_example= normalize.post_normalize_allele(insertion_example)
norm_insertion_example.as_dict()

{'_id': 'ga4gh:VA.J9BMdktHGGjE843oD0T_bwUV6WxojkCW',
 'type': 'Allele',
 'location': {'_id': 'ga4gh:VSL.TMxdXtmi4ctcTRipHMD6py1Nv1kLMyJd',
  'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.Ya6Rs7DHhDeg7YaOSg1EoNi3U_nQ9SvO',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 113901365},
   'end': {'type': 'Number', 'value': 113901365}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'ATA'}}

In [14]:
# Translating the normalized allele into an allele profile
allele_profile_insertion = alleleTranslator.vrs_allele_to_allele_profile(norm_insertion_example)

print(type(allele_profile_insertion))
allele_profile_insertion.model_dump()

<class 'profiles.allele.Allele'>


{'resourceType': 'MolecularDefinition',
 'contained': [{'resourceType': 'MolecularDefinition',
   'id': 'ref-to-nc000001',
   'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
      'code': 'dna',
      'display': 'DNA Sequence'}]},
   'representation': [{'code': [{'coding': [{'system': 'http://www.ncbi.nlm.nih.gov/refseq',
         'code': 'NC_000001.11'}]}]}]}],
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': '#ref-to-nc000001',
     'type': 'MolecularDefinition'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}]}},
     'startQuantity': {'value': 113901365.0},
     'endQuantity': {'value': 113901365.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir

### Example 3

In [15]:
# Creating the VRS object and then normalizing it.
# Example 4 - Substituion origin: "NC_000002.12:g.27453449C>T"
start = 27453448
end = 27453449
refseq = "NC_000002.12"
alt_seq = "T"

interval = models.SequenceInterval(
    start=models.Number(value=start),
    end=models.Number(value=end)
)
location = models.SequenceLocation(
    sequence_id=f"refseq:{refseq}",
    interval=interval
)

state = models.LiteralSequenceExpression(sequence=alt_seq)
sub_example= models.Allele(location=location, state=state)
norm_sub_example= normalize.post_normalize_allele(sub_example)
norm_sub_example.as_dict()

{'_id': 'ga4gh:VA.fXvhngewkkyVwzEeSJRr5tro8Jcol6Q-',
 'type': 'Allele',
 'location': {'_id': 'ga4gh:VSL.nLMbYalHO4OEI2axqkyTMCQxrH98UpDN',
  'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.pnAqCRBrTsUoBghSD1yp_jXWSmlbdh4g',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 27453448},
   'end': {'type': 'Number', 'value': 27453449}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

In [16]:
# Translating the normalized allele into an allele profile
allele_profile_sub_example = alleleTranslator.vrs_allele_to_allele_profile(norm_sub_example)

print(type(allele_profile_sub_example))
allele_profile_sub_example.model_dump()

<class 'profiles.allele.Allele'>


{'resourceType': 'MolecularDefinition',
 'contained': [{'resourceType': 'MolecularDefinition',
   'id': 'ref-to-nc000002',
   'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
      'code': 'dna',
      'display': 'DNA Sequence'}]},
   'representation': [{'code': [{'coding': [{'system': 'http://www.ncbi.nlm.nih.gov/refseq',
         'code': 'NC_000002.12'}]}]}]}],
 'moleculeType': {'coding': [{'system': 'http://hl7.org/fhir/sequence-type',
    'code': 'dna',
    'display': 'DNA Sequence'}]},
 'location': [{'sequenceLocation': {'sequenceContext': {'reference': '#ref-to-nc000002',
     'type': 'MolecularDefinition'},
    'coordinateInterval': {'coordinateSystem': {'system': {'coding': [{'system': 'http://loinc.org',
         'code': 'LA30100-4',
         'display': '0-based interval counting'}]}},
     'startQuantity': {'value': 27453448.0},
     'endQuantity': {'value': 27453449.0}}}}],
 'representation': [{'focus': {'coding': [{'system': 'http://hl7.org/fhir/m

### Conclusion

In this notebook, we demonstrated the translation of GA4GH VRS alleles into HL7 FHIR Allele. We began by creating VRS allele objects, normalizing them, and converting them into FHIR-compliant representations using the VrsFhirAlleleTranslation class. This process enhances interoperability between GA4GH and HL7 FHIR standards, enabling seamless integration of genomic data into healthcare systems. The examples included deletion, insertion, and substitution variants.

We recognize that VRS is continuously evolving, with VRS 2.0 on the horizon. As the new version stabilizes, we plan to assess its impact and potentially refactor our implementation to align with the latest standard, ensuring continued compatibility and functionality.